In [154]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-eligible-dataset/loan-train.csv
/kaggle/input/loan-eligible-dataset/loan-test.csv


### Import Libraries

In [155]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

### Load the Dataset

In [156]:
df=pd.read_csv('/kaggle/input/loan-eligible-dataset/loan-train.csv')

### Check the first 5 rows of dataset

In [157]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


### Check the null values in entire dataset

In [158]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

### Check the shape of dataset

In [159]:
df.shape

(614, 13)

### Check the dtype of each column

In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


## Handle the Missing value

In [161]:
df['Credit_History'].value_counts()

Credit_History
1.0    475
0.0     89
Name: count, dtype: int64

In [162]:
df['Credit_History'].fillna('1.0',inplace=True)

In [163]:
df['Loan_Amount_Term'].value_counts()

Loan_Amount_Term
360.0    512
180.0     44
480.0     15
300.0     13
240.0      4
84.0       4
120.0      3
60.0       2
36.0       2
12.0       1
Name: count, dtype: int64

In [164]:
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].median(),inplace=True)

In [165]:
df['LoanAmount'].value_counts()

LoanAmount
120.0    20
110.0    17
100.0    15
160.0    12
187.0    12
         ..
240.0     1
214.0     1
59.0      1
166.0     1
253.0     1
Name: count, Length: 203, dtype: int64

In [166]:
df['LoanAmount'].fillna(df['LoanAmount'].median(),inplace=True)

In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         614 non-null    float64
 9   Loan_Amount_Term   614 non-null    float64
 10  Credit_History     614 non-null    object 
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(3), int64(1), object(9)
memory usage: 62.5+ KB


In [168]:
df['Self_Employed'].value_counts()

Self_Employed
No     500
Yes     82
Name: count, dtype: int64

In [169]:
df['Self_Employed'].fillna('No',inplace=True)

In [170]:
df['Dependents'].value_counts()

Dependents
0     345
1     102
2     101
3+     51
Name: count, dtype: int64

In [171]:
df['Dependents'] = df['Dependents'].replace('3+', '3').astype(float)

In [172]:
df['Dependents'].fillna(df['Dependents'].median(), inplace=True)

In [173]:
df['Dependents'].value_counts()

Dependents
0.0    360
1.0    102
2.0    101
3.0     51
Name: count, dtype: int64

In [174]:
df['Gender'].value_counts()

Gender
Male      489
Female    112
Name: count, dtype: int64

In [175]:
df['Gender'].fillna(df['Gender'].mode()[0],inplace=True)

In [176]:
df['Gender'].value_counts()

Gender
Male      502
Female    112
Name: count, dtype: int64

#### I handle the missing value with the median or most frequent value.

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             614 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         614 non-null    float64
 4   Education          614 non-null    object 
 5   Self_Employed      614 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         614 non-null    float64
 9   Loan_Amount_Term   614 non-null    float64
 10  Credit_History     614 non-null    object 
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [178]:
# Drop Loan_ID (not useful for prediction)
df.drop(columns=["Loan_ID"], inplace=True)

### Encode the Categorical Variable

In [179]:
# Encode categorical variables
label_cols = ["Gender", "Married", "Education", "Self_Employed", "Credit_History", "Property_Area", "Loan_Status"]
# Convert columns to string and fill NaN with a placeholder (e.g., "Unknown")
df[label_cols] = df[label_cols].astype(str).fillna("Unknown")
for col in label_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

In [180]:
# Define features (X) and target variable (y)
X = df.drop(columns=["Loan_Status"])
y = df["Loan_Status"]

### Split the dataset

In [181]:
# Split into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [182]:
# Add constant for statsmodels
X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

In [183]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    r2_train = model.rsquared
    r2_test = sm.OLS(y_test, X_test).fit().rsquared
    return r2_train, r2_test

## No Elimination

In [184]:
# 1. No Elimination (All Variables)
model_all = sm.OLS(y_train, X_train_sm).fit()
r2_all_train, r2_all_test = evaluate_model(model_all, X_train_sm, X_test_sm, y_train, y_test)

## Backward Elimination

In [185]:
# 2. Backward Elimination
X_opt = X_train_sm.copy()
while True:
    model_be = sm.OLS(y_train, X_opt).fit()
    p_values = model_be.pvalues[1:]  # Exclude intercept
    max_p = p_values.max()
    if max_p > 0.05:
        X_opt.drop(columns=[p_values.idxmax()], inplace=True)
    else:
        break
X_test_be = X_test_sm[X_opt.columns]
r2_be_train, r2_be_test = evaluate_model(model_be, X_opt, X_test_be, y_train, y_test)

## Forward Elimination

In [186]:
# 3. Forward Selection
selected_features = []
remaining_features = list(X_train.columns)
best_r2 = -np.inf
while remaining_features:
    temp_r2 = best_r2
    best_feature = None
    for feature in remaining_features:
        X_temp = sm.add_constant(X_train[selected_features + [feature]])
        model_fs = sm.OLS(y_train, X_temp).fit()
        if model_fs.rsquared_adj > temp_r2:
            temp_r2 = model_fs.rsquared_adj
            best_feature = feature
    if best_feature:
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)
        best_r2 = temp_r2
    else:
        break
X_test_fs = sm.add_constant(X_test[selected_features])
model_fs_final = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
r2_fs_train, r2_fs_test = evaluate_model(model_fs_final, sm.add_constant(X_train[selected_features]), X_test_fs, y_train, y_test)


## Bidirectional

In [187]:
# 4. Stepwise Selection (Forward & Backward)
X_stepwise = sm.add_constant(pd.DataFrame(np.ones(X_train.shape[0]), columns=['const']))
selected_features = []
remaining_features = list(X_train.columns)
while remaining_features:
    best_r2 = -np.inf
    best_feature = None
    for feature in remaining_features:
        X_temp = sm.add_constant(X_train[selected_features + [feature]])
        model_step = sm.OLS(y_train, X_temp).fit()
        if model_step.rsquared_adj > best_r2:
            best_r2 = model_step.rsquared_adj
            best_feature = feature
    if best_feature:
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)
        X_stepwise = sm.add_constant(X_train[selected_features])
    while len(selected_features) > 1:
        model_step = sm.OLS(y_train, X_stepwise).fit()
        p_values = model_step.pvalues[1:]
        max_p = p_values.max()
        if max_p > 0.05:
            selected_features.remove(p_values.idxmax())
            X_stepwise = sm.add_constant(X_train[selected_features])
        else:
            break

In [188]:
X_test_stepwise = sm.add_constant(X_test[selected_features])
model_stepwise = sm.OLS(y_train, X_stepwise).fit()
r2_stepwise_train, r2_stepwise_test = evaluate_model(model_stepwise, X_stepwise, X_test_stepwise, y_train, y_test)

In [189]:
# Collect results
results = {
    "Method": ["All Variables", "Backward Elimination", "Forward Selection", "Stepwise Selection"],
    "Train R²": [r2_all_train, r2_be_train, r2_fs_train, r2_stepwise_train],
    "Test R²": [r2_all_test, r2_be_test, r2_fs_test, r2_stepwise_test],
    "Selected Features": [
        list(X_train.columns),
        list(X_opt.columns[1:]),
        selected_features,
        selected_features,
    ],
}

## Results

In [190]:
df_results = pd.DataFrame(results)
df_results

,Method,Train R²,Test R²,Selected Features
0,All Variables,0.316398,0.331198,"[Gender, Married, Dependents, Education, Self_..."
1,Backward Elimination,0.303279,0.291170,"[Married, Credit_History]"
2,Forward Selection,0.313778,0.309060,"[Credit_History, Married]"
3,Stepwise Selection,0.303279,0.291170,"[Credit_History, Married]"
